<a href="https://colab.research.google.com/github/Raghuram-Veeramallu/Astro_Stat_Project2/blob/development/Project2_Group3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 2
### Group 3


Project Question

In [ ]:
# load data for this instance
!gdown 11uV5nXxSlh0Tg1KF9xI1HYxrTOFEbsbo

Downloading...
From: https://drive.google.com/uc?id=11uV5nXxSlh0Tg1KF9xI1HYxrTOFEbsbo
To: /content/BinaryStarsNGC5053.tsv
100% 2.64k/2.64k [00:00<00:00, 10.5MB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pymc as pm
import arviz as az

In [ ]:
## setting the theme for the grid
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [18]:
df = pd.read_csv('/content/BinaryStarsNGC5053.tsv', sep='\t')

In [19]:
df.head()

,Star,Dist,VR,chi2,P(<chi2)
0,S35,450.4,290.8,1.239,0.8760
1,LY014,498.8,291.7,3.976,0.5528
2,S33,219.2,291.7,0.345,0.5572
3,S37,424.3,291.1,0.125,0.7242
4,LY019,153.5,293.6,1.232,0.7454
